In [ ]:
import numpy as np
import pandas as pd
import requests
import itertools
import os
import json
import random
from sklearn.neighbors.kde import KernelDensity
import numpy as np
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV
from sklearn.cluster import KMeans, AgglomerativeClustering

In [104]:
train = pd.read_json('train.json')
ings = pd.read_csv('ingredients_standardized.csv')
nutrients = ings.loc[:, 'Ingredients':].dropna().values
nutrients_dic = {nutrient[0]: nutrient[1:] for nutrient in nutrients}
recipes_nutrients = []

for recipe in train.ingredients:
    try:
        recipe_nutrients = [nutrients_dic[ing] for ing in recipe]
        recipe_nutrients.sort(key = lambda r: r[0])
        recipes_nutrients.append(np.array(recipe_nutrients))
    except KeyError:
        pass
                            
recipes_nutrients = np.array(recipes_nutrients)
ings.head()

,Unnamed: 0,Ingredients,calories,fat,protein,carbs,saturated fat,cholesterol,sodium,dietary fiber,sugars,potassium,p,serving_weight_grams
0,0,romaine lettuce,17.00,0.30,1.23,3.29,0.04,0.0,8.00,2.10,1.19,247.00,30.00,100.0
1,1,black olives,115.00,10.79,0.79,6.32,1.32,0.0,735.00,3.16,0.00,7.89,2.89,100.0
2,2,grape tomatoes,17.97,0.25,0.89,3.92,0.00,0.0,5.06,1.14,2.66,236.96,24.05,100.0
3,3,garlic,149.00,0.67,6.33,33.00,0.00,0.0,17.00,2.00,1.00,401.00,153.00,100.0
4,4,pepper,250.87,3.04,10.43,63.91,1.30,0.0,20.00,25.22,0.43,1329.13,157.83,100.0


In [2]:
shape_div = {}
for i in range(len(recipes_nutrients)):
    try:
        shape_div[recipes_nutrients[i].shape].append(recipes_nutrients[i])
    except KeyError:
        if(recipes_nutrients[i].shape not in shape_div):
            shape_div[recipes_nutrients[i].shape] = [recipes_nutrients[i]]
shape_div = {key: np.array(shape_div[key]) for key in shape_div.keys()}
for key in sorted(shape_div.keys(), key = lambda r: r[0]):
    print("%s shape has %s meals" % (key, len(shape_div[key])))

(1, 12) shape has 19 meals
(2, 12) shape has 158 meals
(3, 12) shape has 419 meals
(4, 12) shape has 746 meals
(5, 12) shape has 1168 meals
(6, 12) shape has 1495 meals
(7, 12) shape has 1814 meals
(8, 12) shape has 1757 meals
(9, 12) shape has 1711 meals
(10, 12) shape has 1567 meals
(11, 12) shape has 1355 meals
(12, 12) shape has 1144 meals
(13, 12) shape has 926 meals
(14, 12) shape has 678 meals
(15, 12) shape has 461 meals
(16, 12) shape has 314 meals
(17, 12) shape has 242 meals
(18, 12) shape has 169 meals
(19, 12) shape has 82 meals
(20, 12) shape has 65 meals
(21, 12) shape has 37 meals
(22, 12) shape has 14 meals
(23, 12) shape has 14 meals
(24, 12) shape has 6 meals
(25, 12) shape has 2 meals
(26, 12) shape has 2 meals
(27, 12) shape has 1 meals
(28, 12) shape has 2 meals
(29, 12) shape has 1 meals
(30, 12) shape has 1 meals


In [3]:


kdes = {}

# for i in range(1, 31):
#     test_div = []
#     for j in range(len(shape_div[(i, 12)])):
#         to_append = shape_div[(i,12)][j].reshape(1,12*i)
#         test_div.append(shape_div[(i,12)][j].reshape(1,12*i)[0])
#     pca = PCA(n_components = 10, whiten = False)
#     print(np.array(test_div).shape, "is shape")
#     data = pca.fit_transform(np.array(test_div))
#     print(data.shape)
#     params = {'bandwidth': np.logspace(-10, 10, 20)}
#     if len(data) >= 5:
#         grid = GridSearchCV(KernelDensity(kernel = 'gaussian'), params, cv=5)
#         grid.fit(np.array(data))
#         kdes[i] = grid.best_estimator_
#     elif len(data) >= 2:
#         grid = GridSearchCV(KernelDensity(kernel = 'gaussian'), params, cv=len(data))
#         grid.fit(data)
#         kdes[i] = grid.best_estimator_
#     else:
#         pass

for i in range(1, 26):
    test_div = []
    for j in range(len(shape_div[(i, 12)])):
        to_append = shape_div[(i,12)][j]
        if j == 1:
            print("to_append shape ", np.array(to_append).shape)
        test_div.append(np.array(to_append))
    params = {'bandwidth': np.logspace(-10, 10, 100)}
    grid = GridSearchCV(KernelDensity(kernel = 'gaussian'), params, cv=5)
    data = np.array(test_div).reshape(len(shape_div[(i, 12)]), 12 * i)
    print("test_div shape ", data.shape) 
    grid.fit(data)
    kdes[i] = grid.best_estimator_

to_append shape  (1, 12)
test_div shape  (19, 12)
to_append shape  (2, 12)
test_div shape  (158, 24)
to_append shape  (3, 12)
test_div shape  (419, 36)
to_append shape  (4, 12)
test_div shape  (746, 48)
to_append shape  (5, 12)
test_div shape  (1168, 60)
to_append shape  (6, 12)
test_div shape  (1495, 72)
to_append shape  (7, 12)
test_div shape  (1814, 84)
to_append shape  (8, 12)
test_div shape  (1757, 96)
to_append shape  (9, 12)
test_div shape  (1711, 108)
to_append shape  (10, 12)
test_div shape  (1567, 120)
to_append shape  (11, 12)
test_div shape  (1355, 132)
to_append shape  (12, 12)
test_div shape  (1144, 144)
to_append shape  (13, 12)
test_div shape  (926, 156)
to_append shape  (14, 12)
test_div shape  (678, 168)
to_append shape  (15, 12)
test_div shape  (461, 180)
to_append shape  (16, 12)
test_div shape  (314, 192)
to_append shape  (17, 12)
test_div shape  (242, 204)
to_append shape  (18, 12)
test_div shape  (169, 216)
to_append shape  (19, 12)
test_div shape  (82, 228)
to_a

ValueError: Cannot have number of splits n_splits=5 greater than the number of samples: 2.

In [19]:
# def get_score(recipes):
#     print(recipes.shape)
#     kde = kdes[len(recipes[0])]
#     test = []
#     for recipe in recipes:
#         test.append(recipe.reshape(1, 12 * len(recipes[0]))[0])
#     test = np.array(test)
#     print(test.shape)
#     pca = PCA(n_components = 10, whiten = False)
#     data = pca.fit_transform(test)
#     print(data.shape)
#     print(np.array(data).shape)
#     score = kde.score_samples(np.array(data))
#     return score
#     print("index %i score %s" % (i, score))
    
def get_score(recipes):
    kde = kdes[len(recipes[0])]
    data = recipes.reshape(len(recipes), -1)
    print(data.shape)
    score = kde.score_samples(np.array(data))
    return score
    print("recipe with score %s" % (score))
    
def descend(recipe, i):
    reduce_maps = {}
    for i in range(1, i + 1):
        max_ind = 0
        score = 0
        for ind in range(j):
            temp = recipe
            temp.remove(ind)
            poss_score = get_score(np.array([temp]))
            print(poss_score)
            if(poss_score < score):
                score = poss_score
                max_ind = ind
        reduce_maps[j] = (max_ind, score)

In [38]:
for recipe in recipes_nutrients:
    if(len(recipe) == 3):
        print("score %s" %(get_score(np.array([recipe]))))

(3, 12)
(1, 36)
score [-253.56469807]
(3, 12)
(1, 36)
score [-253.62587032]
(3, 12)
(1, 36)
score [-255.68416719]
(3, 12)
(1, 36)
score [-256.86302554]
(3, 12)
(1, 36)
score [-253.58478843]
(3, 12)
(1, 36)
score [-254.60265192]
(3, 12)
(1, 36)
score [-253.04724888]
(3, 12)
(1, 36)
score [-252.82915952]
(3, 12)
(1, 36)
score [-252.93066102]
(3, 12)
(1, 36)
score [-254.86732704]
(3, 12)
(1, 36)
score [-253.38865045]
(3, 12)
(1, 36)
score [-254.01805858]
(3, 12)
(1, 36)
score [-253.09214662]
(3, 12)
(1, 36)
score [-252.81342706]
(3, 12)
(1, 36)
score [-254.37520516]
(3, 12)
(1, 36)
score [-254.47641155]
(3, 12)
(1, 36)
score [-254.50863724]
(3, 12)
(1, 36)
score [-253.08925158]
(3, 12)
(1, 36)
score [-253.66596686]
(3, 12)
(1, 36)
score [-252.94102397]
(3, 12)
(1, 36)
score [-254.06390075]
(3, 12)
(1, 36)
score [-253.23084872]
(3, 12)
(1, 36)
score [-257.06427952]
(3, 12)
(1, 36)
score [-255.68416719]
(3, 12)
(1, 36)
score [-254.4453163]
(3, 12)
(1, 36)
score [-254.8192978]
(3, 12)
(1, 36

score [-255.63472641]
(3, 12)
(1, 36)
score [-255.80092107]
(3, 12)
(1, 36)
score [-253.27161063]
(3, 12)
(1, 36)
score [-253.40854659]
(3, 12)
(1, 36)
score [-257.24304606]


In [20]:
# def rank(arr_to_score):
#     test = {}
#     results = {}
#     for to_score in arr_to_score:
#         try:
#             test[len(to_score)].append(np.array(to_score.reshape(1, 12 * len(to_score)))[0])
#         except KeyError:
#             test[len(to_score)] = []
    
#     for key in test.keys():
#         print(key)
#         pca = PCA(n_components = 10, whiten = False)
#         results[key] = get_score(test[key])
# #         data = pca.fit_transform(np.array(test[key]))
# #         print(data.shape)
# #         if(data.shape[1] < 10):
# #             print("Oh no, reformed shape is", data.shape[1], "with length", len(to_score))
# #         else:
# #             print(np.array(data))
# #             score = kde.get_score(np.array(data))
# #             score.sort()
# #             results[key] = score
    
#     return results

def sorted_scores(recommendations):
    return {score(rec)[0]: rec for rec in np.array(sorted(recommendations, key = lambda r: score(r)))}
        
def score(recommendation):
    nutrients = []
    for ingredient in recommendation:
        if(ingredient in nutrients_dic):
            nutrients.append(nutrients_dic[ingredient])
        else:
            continue
    nutrients = sorted(nutrients, key = lambda c: c[0])
    if(len(nutrients) > 0):
        return get_score(np.array([nutrients]))
    else:
        return 999
    
sorted_scores([recipe for recipe in train.ingredients[:200] if len(recipe) < 6 and len(recipe) > 0])

(1, 48)
(1, 48)
(1, 36)
(1, 48)
(1, 36)
(1, 60)
(1, 24)
(1, 48)
(1, 48)
(1, 48)
(1, 24)
(1, 36)
(1, 24)
(1, 48)
(1, 36)
(1, 48)
(1, 48)
(1, 60)
(1, 24)
(1, 48)
(1, 60)
(1, 48)
(1, 60)
(1, 60)
(1, 60)
(1, 60)
(1, 60)
(1, 48)
(1, 48)
(1, 48)
(1, 48)
(1, 48)
(1, 48)
(1, 48)
(1, 48)
(1, 48)
(1, 48)
(1, 48)
(1, 36)
(1, 36)
(1, 36)
(1, 36)
(1, 24)
(1, 24)
(1, 24)
(1, 24)


{-383.1657670464079: ['yellow corn meal',
  'boiling water',
  'butter',
  'fresh parmesan cheese',
  'sea salt'],
 -383.16466816034097: ['fish sauce',
  'chicken broth',
  'sesame oil',
  'green onions',
  'eggs'],
 -383.060419493975: ['pepper',
  'grated Gruyère cheese',
  'heavy cream',
  'salt',
  'potatoes'],
 -382.77294291418525: ['chopped celery',
  'fire roasted diced tomatoes',
  'andouille sausage',
  'sweet pepper',
  'red kidnei beans, rins and drain'],
 -307.49690060637556: ['sugar', 'hot chili', 'asian fish sauce', 'lime juice'],
 -307.46449090164316: ['refried beans',
  'grated jack cheese',
  'hot sauce',
  'sour cream'],
 -307.46378524686094: ['pizza crust',
  'plum tomatoes',
  'pesto',
  'part-skim mozzarella cheese'],
 -307.0756319954697: ['salt', 'starchy potatoes', 'grated nutmeg', 'flour'],
 -307.0587681175925: ['lemon', 'pesto', 'salmon fillets', 'white wine'],
 -306.8820386088237: ['shredded mozzarella cheese',
  'marinara sauce',
  'fresh basil leaves',
  'pen

In [12]:
{i: kdes[i].get_params()['bandwidth'] for i in range(1, 25)}

{1: 132.19411484660287,
 2: 533.6699231206302,
 3: 335.1602650938834,
 4: 210.49041445120218,
 5: 210.49041445120218,
 6: 335.1602650938834,
 7: 210.49041445120218,
 8: 210.49041445120218,
 9: 335.1602650938834,
 10: 210.49041445120218,
 11: 335.1602650938834,
 12: 210.49041445120218,
 13: 335.1602650938834,
 14: 335.1602650938834,
 15: 533.6699231206302,
 16: 533.6699231206302,
 17: 533.6699231206302,
 18: 533.6699231206302,
 19: 533.6699231206302,
 20: 849.7534359086438,
 21: 335.1602650938834,
 22: 1353.0477745798075,
 23: 1353.0477745798075,
 24: 1353.0477745798075}

In [22]:
def turn_to_query(meal_array):
    query = []
    for meal in meal_array:
        query.append('__'.join(list(map(lambda x: x.replace(' ', '_'), meal))))
    return '___'.join(query)

test_query = [['sweetened condensed milk', 'ice', 'espresso'], ['granulated sugar', 'large egg whites']]
turn_to_query(test_query)

'sweetened_condensed_milk__ice__espresso___granulated_sugar__large_egg_whites'

In [114]:
# new generator idea, hierarchical clustering approach

# params to figure out:
#     number of clusters for a meal
#     set of weights to weight individual contributions to the distance function

# divisive hierarchal clustering on euclidean distance via ward approach

ings = pd.read_csv('ingredients_standardized.csv').dropna()
clustering = AgglomerativeClustering(n_clusters = 100).fit(ings.loc[:, 'calories':'serving_weight_grams'].values)
clustering.children_

array([[    0,   831],
       [    1,   154],
       [   99,   142],
       ...,
       [ 9115, 10656],
       [10652, 10657],
       [10658, 10659]])

In [251]:
ordered_ings = list(nutrients_dic.keys())
ordered_ings_dic = {ordered_ings[i]: i for i in range(len(ordered_ings))}
# matches to ingredient:label dic
recipe_labels = recipe_to_labels(ordered_ings_dic)
# matches to label:array_ingredients dic
labels_dic = {key: [] for key in set(recipe_labels.values())}
for key in recipe_labels.keys():
    labels_dic[recipe_labels[key]].append(key)

# returns ing:cluster_number dic
def recipe_to_labels(recipe):
    return {ing: clustering.labels_[ordered_ings_dic[ing]] for ing in recipe}

# replaces ings of key label with random alternative ings
def substitute_label(key, ings):
    subs = []
    key_array = labels_dic[key]
    print("original ings", ings)
    for ing in ings:
        print(ing in key_array)
        key_array.remove(ing)
    for i in range(len(ings)):
        rand_index = random.randint(0, len(ings) - 1)
        subs.append(key_array[rand_index])
        del key_array[rand_index]
    return subs
            
    
def substitute_recipe(recipe):
    recipe_ing_labels = recipe_to_labels(recipe)
    label_freqs = {key: [ing for ing in recipe_ing_labels.keys() if recipe_ing_labels[ing] == key] for key in set(recipe_ing_labels.values())}
    new_substitute = []
    sorted_keys = sorted(label_freqs.keys(), key = lambda x: len(label_freqs[x]))
    for key in sorted_keys:
        new_substitute += substitute_label(key, label_freqs[key])
    return new_substitute

# labels_dic
old_dfs = pd.read_csv('ingredients.csv').loc[:,'Ingredients':]
old_ings = {ing[0]: ing[1:] for ing in old_dfs.values}
for i in range(4):
    print(sum([old_ings[ing][i] for ing in train.ingredients[8]]))
for i in range(4):
    print(sum([old_ings[ing][i] for ing in substitute_recipe(train.ingredients[8])]))







622.7299999999999
36.17
33.98000000000001
46.199999999999996
original ings ['cheddar cheese']
True
original ings ['corn tortillas']
True
original ings ['ground black pepper']
True
original ings ['jalapeno chilies']
True
original ings ['olive oil']
True
original ings ['chopped cilantro fresh']
True
original ings ['pork']
True
original ings ['salt']
True
original ings ['purple onion', 'fresh pineapple', 'poblano peppers', 'iceberg lettuce', 'lime']
True
True
True
True
True
1080.3299999999997
original ings ['cheddar cheese']
False


ValueError: list.remove(x): x not in list